In [ ]:
import time
import matplotlib.pyplot as plt
import random
import numpy as np
import pickle

import os
from pathlib import Path

print(random.uniform(0.9,1.1))

In [ ]:
current_file_path = Path(get_ipython().run_line_magic('pwd', ''))
root_path = current_file_path.parent
while not any(file.suffix == ".ipynb" for file in root_path.glob("*")):
    root_path = root_path.parent

root_path = str(root_path)
print(root_path)

from keras.models import load_model
model_dir='06_05_154720'#xx_xx_xxxxxx
model=load_model(os.path.join(root_path,'model_training','model_record',model_dir,'model.h5'))



In [ ]:
import sys
import random
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QWidget, QGridLayout, QMessageBox
from PyQt5.QtGui import QPainter, QPen, QColor, QFont, QPainterPath
from PyQt5.QtCore import QTimer, Qt, QPoint
import numpy as np
import pickle
from pathlib import Path
import os
import math
# probs=np.full((2, 225),0)
probs = [0 for _ in range(225)]
a=[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],False]

class DrawingWidget(QWidget):
    
    def __init__(self, board_size):
        super().__init__()  
        self.board_size = board_size
        self.show_AI_ana=True
    def paintEvent(self, event):
        super().paintEvent(event)  # 調用父類的 paintEvent 方法
        painter = QPainter(self)
        pen = QPen(Qt.black, 1) 
        for col in range(15):
            painter.setPen(pen)
            painter.drawLine(29+col*40, 24, 29+col*40, 584)

        for row in range(15):
            painter.setPen(pen)
            painter.drawLine(29, 24+row*40, 589, 24+row*40)

        # 繪制黑色圆形
        pen = QPen(Qt.black)
        painter.setPen(pen)
        painter.setBrush(Qt.black)
        radius = 5
        
        center1 = QPoint(149, 145)
        center2 = QPoint(469, 145)
        center3 = QPoint(309, 304)
        center4 = QPoint(149, 465)
        center5 = QPoint(469, 465)
        for center in [center1,center2,center3,center4,center5]:
            painter.drawEllipse(center, radius, radius)

        painter2=QPainter(self)
        painter2.setRenderHint(QPainter.Antialiasing)  # 抗锯齿效果
          # 设置画刷颜色为红色
        painter2.setRenderHint(QPainter.Antialiasing, True)
        painter2.setPen(QPen(Qt.NoPen))

        top_n_move = sorted(range(len(probs)), key=lambda i: probs[i], reverse=True)[:5]
        top= np.argmax(probs)

        for row in range(self.board_size):
            for col in range(self.board_size):
                opa=probs[row*15+col]
                painter2.setOpacity(math.pow(opa,0.3))
                if(row*15+col==top):
                    painter2.setBrush(QColor(255, 255, 0))
                elif(row*15+col in top_n_move):
                    painter2.setBrush(QColor(20, 255, 100))
                else:
                    painter2.setBrush(QColor(20, 100, 255))
                rect=painter2.drawRect(row*40+15, col*40+9, 30, 30)

         # 绘制矩形，左上角坐标为(50, 50)，宽度为100，高度为100
class MainWindow(QMainWindow):
    def get_board(self):
        return self.board
    def __init__(self):
        super().__init__()
        self.setWindowTitle('五子棋')
        self.board_size = 15  # 棋盤尺寸
        self.player = None  # 當前玩家（黑子或白子）
        self.ai_player = None  # AI玩家（黑子或白子）
        self.board = [[-1] * self.board_size for _ in range(self.board_size)]  # 棋盤狀態
        self.game_over = False  # 遊戲結束標記
        self.player_can_play = True  # 玩家是否可以點擊按钮
        self.counter=0
        self.step_cnt=0
        self.state_record=np.full((15,15),-1)
        self.init_ui(startup=True)
        width = 670
        height = 670
        self.resize(width, height)
    
    def init_ui(self,startup):
        central_widget = QWidget()
        layout = QGridLayout()  # 使用QGridLayout作為主布局
        central_widget.setLayout(layout)

        self.status_label = QLabel()
        layout.addWidget(self.status_label, 0, 0)  # 放置在第0行第0列

        buttonM = QPushButton("AI 分析")
        buttonM.clicked.connect(self.ai_show_clicked)
        buttonM.setFixedSize(50, 30)  # 設置按鈕的固定大小
        layout.addWidget(buttonM, 0, 1)  # 放置在第0行第1列
        button_reset = QPushButton("reset")
        button_reset.clicked.connect(self.reset)
        button_reset.setFixedSize(50, 30)  # 設置按鈕的固定大小
        layout.addWidget(button_reset, 0, 2)
        self.drawing_widget = DrawingWidget(self.board_size)
        layout.addWidget(self.drawing_widget, 1, 0, 1, 2)  # 放置在第1行，跨越2列
        
        self.buttons = []
        button_size = 40
        for row in range(self.board_size):
            row_buttons = []
            for col in range(self.board_size):
                
            
                button = QPushButton(self.drawing_widget)
                button.setFixedSize(button_size, button_size)
                button.clicked.connect(lambda checked, row=row, col=col: self.player_on_button_clicked(row, col))
                button.setStyleSheet("background-color: transparent; border: none;")  # 將按鈕設置為透明
                button.move((row * button_size)+10, (col * button_size)+4 )
                
                font = QFont()
                font.setPointSize(20)  # 設定字體大小為 20
                button.setFont(font)  # 設定字體
                if(self.board[row][col]=='黑'):
                    button.setText("⚫")
                elif(self.board[row][col]=='白'):
                    button.setText("⚪")
                row_buttons.append(button)
            
            self.buttons.append(row_buttons)
            
        self.setCentralWidget(central_widget)
        self.setStyleSheet("background-color: #D2B48C;")
        if(startup):
            self.choose_player()
    def reset(self):
        self.board = [[-1] * self.board_size for _ in range(self.board_size)]
        self.game_over = False  # 遊戲結束標記
        self.counter=0
        self.step_cnt=0
        self.state_record=np.full((15,15),-1)
        self.update_prob()
        self.init_ui(startup=True)
    def choose_player(self):
        msg_box = QMessageBox()
        msg_box.setWindowTitle('選擇棋子')
        msg_box.setText('請選擇您要執的棋子：')
        black_button = msg_box.addButton('黑子', QMessageBox.YesRole)
        white_button = msg_box.addButton('白子', QMessageBox.NoRole)
        msg_box.exec_()
        if msg_box.clickedButton() == black_button:
            self.player = '黑'
            self.ai_player = '白'
        elif msg_box.clickedButton() == white_button:
            self.player = '白'
            self.ai_player = '黑'
        self.status_label.setText(f'當前玩家：{self.player}')
        
        if self.ai_player =='黑':
           self.player = self.ai_player
           print("do AI play")
           self.ai_play()
    def player_on_button_clicked(self, row, col):
        
        if self.game_over or self.board[row][col] != -1 or not self.player_can_play:
            return
        self.state_record[row][col]=self.step_cnt
        self.step_cnt=self.step_cnt+1
        self.board[row][col] = self.player
        if self.player == '黑':
            self.buttons[row][col].setText("⚫")
        else:
            self.buttons[row][col].setText("⚪")

        self.update_prob()#for AI

        if self.check_win(row, col):
            self.game_over = True
            self.status_label.setText(f'{self.player}獲勝！')
        else:
            self.player = '白' if self.player == '黑' else '黑'
            self.status_label.setText(f'當前玩家：{self.player}')
            if not self.game_over and self.player == self.ai_player:
                self.player_can_play = False  
                self.ai_play()
    def ai_show_clicked(self):
        if(a[1]==False):
            a[1]=True
            self.update_prob()
        else:
            a[1]=False
            self.init_ui(startup=False)
    def update_prob(self):
        if(a[1]):
            available_positions = []
            for row in range(self.board_size):
                for col in range(self.board_size):
                    if self.board[row][col] != -1:
                        available_positions.append(row*15+col)
            _,_,p=self.get_ai_play(available_positions)
            for i in range(225):
                probs[i]=p[i]
            self.init_ui(startup=False)
    def get_ai_play(self,available_positions):
        state=self.state_record
        x=np.full((5,15,15),0)
        p1=(state+1)%2
        all_move=np.where(state<0,0,1)
        p2=all_move-p1
        if(int(np.max(state))%2==0):
            x[2]=p2
            x[3]=p1
        else:
            x[3]=p2
            x[2]=p1
            x[0]+=1
        lastmove = np.where(state==self.step_cnt-1)
        if(self.step_cnt!=0):
            x[4][lastmove]=1
        x[1]=1-x[3]-x[2]
        x= np.expand_dims(x, axis=0)
        prediction=model.predict(x)
        prob=prediction[0].flatten()
        # print(x)
        prob[available_positions]=0
        move=np.argmax(prob)
        print(move)
        return int(move/15),move%15,prob
    def ai_play(self):
        available_positions = []
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.board[row][col] != -1:
                    available_positions.append(row*15+col)
        
        if len(available_positions)<225:
            print("get AI move")
            row, col ,_= self.get_ai_play(available_positions)
            # get AI move
            self.board[row][col] = self.ai_player
            self.state_record[row][col]=self.step_cnt
            self.step_cnt=self.step_cnt+1
            if self.ai_player == '黑':
                self.buttons[row][col].setText("⚫")
            else:
                self.buttons[row][col].setText("⚪")
            self.update_prob()
            if self.check_win(row, col):
                self.game_over = True
                self.status_label.setText(f'{self.player}獲勝！')
            else:
                # self.ai_play()
                # self.ai_player = '白' if self.ai_player == '黑' else '黑'
                # self.status_label.setText(f'當前玩家：{self.ai_player}')

                self.player_can_play = True  # 允许玩家点击按钮
                self.player = '白' if self.player == '黑' else '黑'
                self.status_label.setText(f'當前玩家：{self.player}')

    def check_win(self, row, col):
        # 檢查行
        for c in range(self.board_size - 5):
            if self.board[row][c] == self.board[row][c+1] == self.board[row][c+2] == self.board[row][c+3] ==  self.board[row][c+4] == self.board[row][c+5] == self.player:
                return False

        # 檢查列
        for r in range(self.board_size - 5):
            if self.board[r][col] == self.board[r+1][col] == self.board[r+2][col] == self.board[r+3][col] == self.board[r+4][col] ==  self.board[r+5][col] == self.player:
                return False

        # 檢查正對角線
        for i in range(self.board_size - 5):
            if row - i >= 0 and col - i >= 0 and row - i + 5 < self.board_size and col - i + 5 < self.board_size:
                if self.board[row-i][col-i] == self.board[row-i+1][col-i+1] == self.board[row-i+2][col-i+2] == self.board[row-i+3][col-i+3] == self.board[row-i+4][col-i+4] == self.board[row-i+5][col-i+5] == self.player:
                    return False

        # 檢查負對角線
        for i in range(self.board_size - 5):
            if row + i < self.board_size and col - i >= 0 and row + i - 5 >= 0 and col - i + 5 < self.board_size:
                if self.board[row+i][col-i] == self.board[row+i-1][col-i+1] == self.board[row+i-2][col-i+2] == self.board[row+i-3][col-i+3] == self.board[row+i-4][col-i+4] == self.board[row+i-5][col-i+5] == self.player:
                    return False
                
        # 檢查行        
        for c in range(self.board_size - 4):
            if self.board[row][c] == self.board[row][c+1] == self.board[row][c+2] == self.board[row][c+3] == self.board[row][c+4] == self.player:
                return True

        # 檢查列
        for r in range(self.board_size - 4):
            if self.board[r][col] == self.board[r+1][col] == self.board[r+2][col] == self.board[r+3][col] == self.board[r+4][col] == self.player:
                return True

        # 檢查正對角線
        for i in range(self.board_size - 4):
            if row - i >= 0 and col - i >= 0 and row - i + 4 < self.board_size and col - i + 4 < self.board_size:
                if self.board[row-i][col-i] == self.board[row-i+1][col-i+1] == self.board[row-i+2][col-i+2] == self.board[row-i+3][col-i+3] == self.board[row-i+4][col-i+4] == self.player:
                    return True

        # 檢查負對角線
        for i in range(self.board_size - 4):
            if row + i < self.board_size and col - i >= 0 and row + i - 4 >= 0 and col - i + 4 < self.board_size:
                if self.board[row+i][col-i] == self.board[row+i-1][col-i+1] == self.board[row+i-2][col-i+2] == self.board[row+i-3][col-i+3] == self.board[row+i-4][col-i+4] == self.player:
                    return True

        return False


app = QApplication(sys.argv)
window = MainWindow()
window.show()
sys.exit(app.exec_())